# Compare sic codes and descriptions from various sources
SIC codes and descriptions are available from a variety of sources. In this document, I compare lists of four-digit SIC codes from three different sources:

1. [OSHA website](https://www.osha.gov/pls/imis/sic_manual.html)
1. [SEC website](https://www.sec.gov/info/edgar/siccodes.htm)
1. [Scientific Telephone Samples website](http://www.stssamples.com/sic-code.asp)

### Key findings:
* The SEC provides a different set of four-digit SIC codes:
    * It provides fewer overall codes (444 codes, compared to OSHA's 1005)
    * Some of the SIC codes it provides cannot be found in OSHA list - these are likely various aggregations of underlying four-digit SIC codes
* The reference list of SIC codes shares all codes in common with OSHA, though some descriptions differ sightly

## Setup

In [2]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join('../scrape_sic')))

import pickle
import pandas as pd

import scrape_sic_osha as scrape_osha
import scrape_sic_sec as scrape_sec

import nltk
from __future__ import division

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/Ryan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Ryan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

##  Compare OSHA to SEC

### Clean and merge data

In [3]:
# Read OSHA data
osha_fname = 'osha_combined.csv'
try:
    osha = pd.read_csv(osha_fname)
except IOError:
    scrape_osha.get_sic_all()
    osha = pd.read_csv(osha_fname)
len(osha)

1005

In [4]:
# Read SEC data
sec_fname = 'sec_combined.csv'
try:
    sec = pd.read_csv(sec_fname)
except IOError:
    scrape_sec.save_sic_sec()
    sec = pd.read_csv(sec_fname)
len(sec)

444

In [5]:
# Merge OSHA and SEC data
inner = osha.merge(sec, how='inner', on='SIC4_cd')
len(inner)

272

### Compare descriptions for each four-digit SIC code in common

In [6]:
osha_desc = list(inner.SIC4_desc.str.lower().str.strip())
len(osha_desc)

272

In [7]:
sec_desc = list(inner.industry_title.str.lower().str.strip())
len(sec_desc)

272

In [8]:
match = []
for i in range(0, len(inner)):
    
    # Identify direct matches
    match_ind = sec_desc[i] == osha_desc[i]
    if not(match_ind):
        
        # Where not a direct match count the number of indirect matches
        tokens_taged = nltk.pos_tag(nltk.word_tokenize(osha_desc[i]))
        osha_words = [word[0] for word in tokens_taged]
        sec_words = [word[0] for word in nltk.pos_tag(nltk.word_tokenize(sec_desc[i]))]
        
        word_matches = [word[0] in sec_words for word in tokens_taged if word[1] != 'CC']
        match_rate = sum(word_matches)/len(word_matches)
        if match_rate > 0.3:
            match_ind = True
    match.append(match_ind)

### Summary
Nearly all of the shared four-digit SIC codes from the OSHA and SEC lists shared a similar description, on the basis of a direct match or an indirect (30% or more of words in common, excluding coordinating-conjunctions) match. Of mismatches, most can be attributed to punctuation, grammar or syntax (i.e., as opposed to reference to an entirely different industry).

In [22]:
# Identify match rate
sum(match)/len(inner)

0.9926470588235294

In [9]:
# Identify specific mismatches
inner[[not(m) for m in match]]

,SIC4_cd,SIC4_desc,ind_cd,ind_desc,maj_cd,maj_desc,div_cd,div_desc,AD_office,industry_title
94,3585,Air-Conditioning and Warm Air Heating Equipmen...,358,Refrigeration And Service Industry Machinery,35,Industrial And Commercial Machinery And Comput...,D,Manufacturing,6,AIR-COND & WARM AIR HEATG EQUIP & COMM & INDL ...
176,5072,Hardware,507,"Hardware, And Plumbing And Heating Equipment",50,Wholesale Trade-durable Goods,F,Wholesale Trade,6,WHOLESALE-HARDWARE


## Compare OSHA to benchmark

### Clean and merge data

In [26]:
benchmark =  pd.read_csv('tests/ref_list.csv')
len(benchmark)

1004

In [37]:
benchmark.columns = ['SIC4_cd', 'SIC4_desciption']
benchmark.head()

,SIC4_cd,SIC4_desciption
0,111,WHEAT
1,112,RICE
2,115,CORN
3,116,SOYBEANS
4,119,"CASH GRAINS, NEC"


### Compare descriptions for each four-digit SIC code in common

In [38]:
# Merge OSHA and benchmark data
inner = osha.merge(benchmark, how='inner', on='SIC4_cd')
len(inner)

1004

In [40]:
osha_desc = list(inner.SIC4_desc.str.lower().str.strip())
len(osha_desc)

1004

In [41]:
benchmark_desc = list(inner.SIC4_desciption.str.lower().str.strip())
len(benchmark_desc)

1004

In [64]:
match = []
for i in range(0, len(inner)):
    
    # Count direct matches
    match_ind = benchmark_desc[i] == osha_desc[i]
    if not(match_ind):
        
        # Where not a direct match count the number of indirect matches
        tokens_taged = nltk.pos_tag(nltk.word_tokenize(benchmark_desc[i].replace(', nec', '')))
        osha_words = [word[0] for word in nltk.pos_tag(nltk.word_tokenize(osha_desc[i]))]
        
        word_matches = [word[0] in osha_words for word in tokens_taged if word[1] != 'CC']
        match_rate = sum(word_matches)/len(word_matches)
        if match_rate > 0.3:
            match_ind = True
    match.append(match_ind)

### Summary
Nearly all of the shared four-digit SIC codes from the OSHA and SEC lists shared a similar description, on the basis of a direct match or an indirect (30% or more of words in common, excluding coordinating-conjunctions) match. Of mismatches, most can be attributed to punctuation, grammar or syntax (i.e., as opposed to reference to an entirely different industry).

In [65]:
# Identify match rate
print sum(match)/len(inner)

0.987051792829


In [66]:
# Identify specific mismatches
inner[[not(m) for m in match]]

,SIC4_cd,SIC4_desc,ind_cd,ind_desc,maj_cd,maj_desc,div_cd,div_desc,SIC4_desciption
89,1521,General Contractors-Single-Family Houses,152,General Building Contractors-residential,15,Building Construction General Contractors And ...,C,Construction,SINGLE-FAMILY HOUSING CONSTRUCTION
90,1522,"General Contractors-Residential Buildings, Oth...",152,General Building Contractors-residential,15,Building Construction General Contractors And ...,C,Construction,"RESIDENTIAL CONSTRUCTION, NEC"
93,1542,"General Contractors-Nonresidential Buildings, ...",154,General Building Contractors-nonresidential,15,Building Construction General Contractors And ...,C,Construction,"NONRESIDENTIAL CONSTRUCTION, NEC"
179,2261,Finishers of Broadwoven Fabrics of Cotton,226,"Dyeing And Finishing Textiles, Except Wool Fab...",22,Textile Mill Products,D,Manufacturing,"FINISHING PLANTS, COTTON"
180,2262,Finishers of Broadwoven Fabrics of Manmade Fib...,226,"Dyeing And Finishing Textiles, Except Wool Fab...",22,Textile Mill Products,D,Manufacturing,"FINISHING PLANTS, MANMADE"
181,2269,"Finishers of Textiles, Not elsewhere Classified",226,"Dyeing And Finishing Textiles, Except Wool Fab...",22,Textile Mill Products,D,Manufacturing,"FINISHING PLANTS, NEC"
215,2392,"House furnishing, Except Curtains and Draperies",239,Miscellaneous Fabricated Textile Products,23,Apparel And Other Finished Products Made From ...,D,Manufacturing,"HOUSEHOLD FURNISHINGS, NEC"
291,2833,Medicinal Chemicals and Botanical Products,283,Drugs,28,Chemicals And Allied Products,D,Manufacturing,MEDICINALS AND BOTANICALS
371,3315,Steel Wiredrawing and Steel Nails and Spikes,331,"Steel Works, Blast Furnaces, And Rolling And F...",33,Primary Metal Industries,D,Manufacturing,STEEL WIRE AND RELATED PRODUCTS
372,3316,"Cold-Rolled Steel Sheet, Strip, and Bars",331,"Steel Works, Blast Furnaces, And Rolling And F...",33,Primary Metal Industries,D,Manufacturing,COLD FINISHING OF STEEL SHAPES
